# Step 1: Quality Control and pilot selection

In [1]:
import os
import glob
from pathlib import Path
#from dotenv import load_dotenv
import pandas as pd
import numpy as np
# import dask.dataframe as dd
# import dask.bag as db

import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
nat_img_dir='/nese/mit/group/sig/projects/naturalistic/nat_img/'


## load saved qc csvs from MRI-QC

can skip this if re-running and just load the new saved csv

In [2]:
movieDM_qc = pd.read_csv(f"{nat_img_dir}sourcedata/data/HBN/phenotype/movieDM_qc.csv")
movieTP_qc = pd.read_csv(f"{nat_img_dir}sourcedata/data/HBN/phenotype/movieTP_qc.csv")
rest1_qc = pd.read_csv(  f"{nat_img_dir}sourcedata/data/HBN/phenotype/rest1_qc.csv"  )
rest2_qc = pd.read_csv(  f"{nat_img_dir}sourcedata/data/HBN/phenotype/rest2_qc.csv"  )

In [4]:
#combine all of them into one and format
import ast
all_qc = pd.concat([movieDM_qc, movieTP_qc, rest1_qc, rest2_qc])

all_qc['task'] = all_qc['task'].str.replace('run-1', '1')
all_qc['task'] = all_qc['task'].str.replace('run-2', '2')
all_qc['task'] = all_qc['task'].str.replace('task-movieTP', 'movieTP')
all_qc['task'] = all_qc['task'].str.replace('task-movieDM', 'movieDM')
all_qc.rename(columns={'task': 'run_bad'}, inplace=True)

# all_qc['subj_id'] = all_qc['subj_id'].str.replace('sub-', '')
# all_qc.rename(columns={'subj_id': 'sub'}, inplace=True)

# Assuming 'all_qc' is your DataFrame and 'bids_meta' is your column
all_qc['bids_meta'] = all_qc['bids_meta'].apply(ast.literal_eval)
new_columns = all_qc['bids_meta'].apply(pd.Series)
# Now, 'new_columns' is a DataFrame where each key-value pair in the dictionaries
# are separated into columns and values. You can add these new columns to your original DataFrame.
all_qc = pd.concat([all_qc, new_columns[['session_id', 'subject_id', 'task_id']]], axis=1)

all_qc.rename(columns={'task_id': 'task'}, inplace=True)
all_qc.rename(columns={'session_id': 'ses'}, inplace=True)
all_qc.rename(columns={'subject_id': 'sub'}, inplace=True)



#filter them 1) leniently and 2) more stringently and save these 2 filters as binary columns
all_qc['relaxed_fd50'] = np.where(all_qc['fd_perc'] < 50, 1, 0)
all_qc['dorit_fd40'] = np.where(all_qc['fd_perc'] < 40, 1, 0)
#save this file
all_qc.to_csv('../data/all_qc.csv', index=False)

In [6]:
#these subjects were ppotentially erroneously exlcuded before due to incorrect task name

acq=all_qc[all_qc['run_bad'].str.contains('acq')]

# unique_values = acq['run_bad'].unique()
# print(unique_values)
acq

,aor,aqi,bids_meta,dummy_trs,dvars_nstd,dvars_std,dvars_vstd,efc,fber,fd_mean,...,summary_fg_p95,summary_fg_stdv,tsnr,subj_id,run_bad,ses,sub,task,relaxed_fd50,dorit_fd40
7,0.001119,0.011860,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",2,45.495832,1.128268,0.999489,0.515749,1154.753662,0.255216,...,3050.532227,613.563923,31.289108,sub-NDARAB514MAJ,acq-VARIANTObliquity,HBNsiteCBIC,NDARAB514MAJ,movieDM,0,0
54,0.004310,0.056349,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",0,64.236804,0.925733,0.884248,0.527148,1526.953003,1.341207,...,537.134644,101.090118,13.010549,sub-NDARAM848GTE,acq-VARIANTObliquity,HBNsiteRU,NDARAM848GTE,movieDM,0,0
55,0.002965,0.008608,"{'AcquisitionMatrixPE': 90, 'BandwidthPerPixel...",2,41.896795,1.144066,1.017792,0.449267,1805.770142,0.381601,...,293.353333,62.713866,36.707233,sub-NDARAM873GAC,acq-ABCD,HBNsiteCBIC,NDARAM873GAC,movieDM,0,0
58,0.000710,0.007471,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",2,37.603848,1.093142,0.991591,0.529873,1019.135620,0.147037,...,3089.807617,617.898815,38.125343,sub-NDARAN524ZK6,acq-VARIANTObliquity,HBNsiteCBIC,NDARAN524ZK6,movieDM,1,1
75,0.007404,0.030756,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",0,53.204824,0.948002,0.938388,0.518920,1001.397522,0.685261,...,2887.676025,569.012096,19.319048,sub-NDARAR545KXM,acq-VARIANTObliquity,HBNsiteCBIC,NDARAR545KXM,movieDM,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4510,0.004520,0.008741,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",4,40.185350,1.091201,1.014881,0.496283,1330.666016,0.381379,...,819.089417,188.501709,31.880240,sub-NDARZE389XF0,acq-VARIANTObliquity,HBNsiteRU,NDARZE389XF0,movieTP,0,0
4513,0.000370,0.006534,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",2,37.161979,1.037936,0.997094,0.518176,880.294189,0.184468,...,2577.193359,496.596533,39.070944,sub-NDARZE963MEU,acq-VARIANTObliquity,HBNsiteCBIC,NDARZE963MEU,movieTP,1,0
4541,0.001431,0.007826,"{'AcquisitionMatrixPE': 84, 'BandwidthPerPixel...",3,47.174404,1.338811,1.009905,0.529563,940.764954,0.131134,...,3145.234863,653.849324,36.451010,sub-NDARZL799MFW,acq-VARIANTObliquity,HBNsiteCBIC,NDARZL799MFW,movieTP,1,1
4553,0.001961,0.003765,"{'AcquisitionMatrixPE': 90, 'BandwidthPerPixel...",4,24.850039,1.028776,1.022217,0.465242,1400.076294,0.092529,...,329.270935,77.384349,46.804991,sub-NDARZN677EYE,acq-ABCD,HBNsiteCBIC,NDARZN677EYE,movieTP,1,1


## load fixed qc dataframe

In [3]:
all_qc = pd.read_csv(  f"../data/all_qc.csv"  )

## Load demographic (dx) data

In [4]:
def get_imaging_site(identifier):
    path = f'/nese/mit/group/sig/projects/hbn/hbn_bids/sub-{identifier}/'
    if os.path.isdir(path + 'ses-HBNsiteRU'):
        return 'HBNsiteRU'
    elif os.path.isdir(path + 'ses-HBNsiteCBIC'):
        return 'HBNsiteCBIC'
    elif os.path.isdir(path + 'ses-HBNsiteSI'):
        return 'HBNsiteSI'
    elif os.path.isdir(path + 'ses-HBNsiteCUNY'):
        return 'HBNsiteCUNY'
    else:
        return None
        
dx_final=pd.read_csv(f'{nat_img_dir}sourcedata/data/HBN/phenotype/parsed/dx_onehot.csv')
dx_final['imaging_site'] = dx_final['Identifiers'].apply(get_imaging_site)

filtered_df = dx_final[dx_final['Autism Spectrum Disorder'] == 1]
asd_df = filtered_df[['Identifiers','imaging_site']].rename(columns={'Identifiers': 'subj_id'})
asd_df['subj_id'] = 'sub-' + asd_df['subj_id'].astype(str)

filtered_df = dx_final[dx_final['No Diagnosis Given'] == 1]
nt_df = filtered_df[['Identifiers','imaging_site']].rename(columns={'Identifiers': 'subj_id'})
nt_df['subj_id'] = 'sub-' + nt_df['subj_id'].astype(str)

filtered_df = dx_final[(dx_final['No Diagnosis Given'] == 0) & (dx_final['Autism Spectrum Disorder'] == 0)]
pilot_df = filtered_df[['Identifiers','imaging_site']].rename(columns={'Identifiers': 'subj_id'})
pilot_df['subj_id'] = 'sub-' + pilot_df['subj_id'].astype(str)

In [5]:
dx_final

,Identifiers,Age,Sex,Site,ADHD-Combined Type,ADHD-Hyperactive/Impulsive Type,ADHD-Inattentive Type,Acute Stress Disorder,Adjustment Disorders,Agoraphobia,...,Unspecified Anxiety Disorder,Unspecified Attention-Deficit/Hyperactivity Disorder,Unspecified Bipolar and Related Disorder,Unspecified Depressive Disorder,Unspecified Feeding or Eating Disorder,Unspecified Intellectual Disability,Unspecified Neurodevelopmental Disorder,Unspecified Tic Disorder,Unspecified Trauma- and Stressor-Related Disorder,imaging_site
0,NDARAA075AMK,6.728040,female,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteSI
1,NDARAA112DMH,5.545744,male,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteRU
2,NDARAA117NEJ,7.475929,male,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteRU
3,NDARAA306NT2,21.216746,female,3,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteRU
4,NDARAA358BPN,11.853296,male,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4762,NDARZT957CWG,16.034565,male,3,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteCBIC
4763,NDARZU401RCU,17.663586,female,3,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteCBIC
4764,NDARZV458LDP,15.763175,female,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,HBNsiteCUNY
4765,NDARZX745YLZ,16.288501,male,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,None


### filter the subjects by task (dm), site (ru), quality (relaxed), dx for matching

In [6]:
'''
I want a dataframe which has:

from all_qc
- only the relaxed filtered subjects
- only task==DM

from dx_final:
- a 'case' column where ASD (1) and controls (0), others (-1)
- only ses==RU
'''

dx_final_ru = dx_final[dx_final['imaging_site'] == 'HBNsiteRU']

def assign_class(row):
    if row['Autism Spectrum Disorder'] == 1:
        return 1
    elif row['No Diagnosis Given'] == 1:
        return 0
    else:
        return -1

dx_final_ru['class'] = dx_final_ru.apply(assign_class, axis=1)

all_qc_ru_dm = all_qc[all_qc['ses'] == 'HBNsiteRU']
all_qc_ru_dm = all_qc_ru_dm[all_qc_ru_dm['task'] == 'movieDM']
all_qc_ru_dm = all_qc_ru_dm[all_qc_ru_dm['relaxed_fd50'] == 1]

#get the common subjects between these dataframes
dx_final_ru = dx_final_ru[dx_final_ru['Identifiers'].isin(all_qc_ru_dm['sub'])]


/tmp/ipykernel_837465/837493429.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dx_final_ru['class'] = dx_final_ru.apply(assign_class, axis=1)


### match the ASD to NT by age and sex and then match the matches to non-asd, non-NT for RU pilot subset

In [10]:
# Load the data into a pandas dataframe
df = dx_final_ru[['Identifiers', 'Age', 'Sex', 'class','imaging_site']]

#df = df.loc[(df['imaging_site'] == 'RU') | (df['imaging_site'] == 'CBIC')]


# Separate cases and controls based on the `Group` variable
cases = df[df['class'] == 0]
controls = df[df['class'] == 1]

# Rename the variables for cases and controls to distinguish them
cases = cases.rename(columns={'Age': 'Age_case'})
controls = controls.rename(columns={'Age': 'Age_control'})

# Join the cases and controls based on age and sex
matched = pd.merge(cases, controls, on=['Sex'])
matched = matched[(matched['Age_control'] - 1 <= matched['Age_case']) & (matched['Age_case'] <= matched['Age_control'] + 1)]

# Randomly select one match if there are more
matched = matched.sample(frac=1).groupby('Identifiers_x').head(1)
matched = matched.sample(frac=1).groupby('Identifiers_y').head(1)

###
#Now using these matched asd and controls, make a matched pilot subset from the non-control non-asd population
###


# Separate cases and controls based on the `Group` variable
controls = df[df['class'] == -1]

# Rename the variables for cases and controls to distinguish them
#cases = cases.rename(columns={'Age': 'Age_case'})
cases= matched[['Identifiers_x', 'Age_case', 'Sex', 'class_x','imaging_site_x']]
cases.rename(columns={'Identifiers_x': 'Identifiers'}, inplace=True)
cases.rename(columns={'class_x': 'class'}, inplace=True)
cases.rename(columns={'imaging_site_x': 'imaging_site'}, inplace=True)


controls = controls.rename(columns={'Age': 'Age_control'})

# Join the cases and controls based on age and sex
matched2 = pd.merge(cases, controls, on=['Sex'])
matched2 = matched2[(matched2['Age_control'] - 1 <= matched2['Age_case']) & (matched2['Age_case'] <= matched2['Age_control'] + 1)]

# Randomly select one match if there are more
matched2 = matched2.sample(frac=1).groupby('Identifiers_x').head(1)
matched2 = matched2.sample(frac=1).groupby('Identifiers_y').head(1)

pilots = matched2[['Identifiers_y', 'Age_control', 'Sex', 'class_y','imaging_site_y']]

/tmp/ipykernel_837465/2201919577.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases.rename(columns={'Identifiers_x': 'Identifiers'}, inplace=True)
/tmp/ipykernel_837465/2201919577.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases.rename(columns={'class_x': 'class'}, inplace=True)
/tmp/ipykernel_837465/2201919577.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases.rename(columns={'imaging_site_x': 'imaging_site'}

In [11]:
pilots

,Identifiers_y,Age_control,Sex,class_y,imaging_site_y
5095,NDARCL016NHB,11.177503,female,-1,HBNsiteRU
4457,NDARPT910NEE,11.500570,male,-1,HBNsiteRU
2285,NDARCA690EBC,12.283709,male,-1,HBNsiteRU
3743,NDARRW141HUT,14.876454,male,-1,HBNsiteRU
1625,NDARUB231LHN,6.413187,female,-1,HBNsiteRU
2760,NDARWP157RYM,6.081907,female,-1,HBNsiteRU
1031,NDARDF568GL5,7.853524,female,-1,HBNsiteRU
3052,NDARXR637JER,12.475359,female,-1,HBNsiteRU
3593,NDARZH672BAM,7.333105,male,-1,HBNsiteRU
2532,NDARJH729HXT,10.818845,male,-1,HBNsiteRU


In [21]:
# pilot_subjects=pd.read_csv('../data/pilots_ru_dm.csv') # load pilot subjects
import nibabel as nb
subjects=[]
for sub in list(pilots['Identifiers_y']):
    
    im_file = f'/nese/mit/group/sig/projects/hbn/hbn_bids/derivatives/xcp_d_0.7.1/sub-{sub}/ses-HBNsiteRU/func/sub-{sub}_ses-HBNsiteRU_task-movieDM_space-fsLR_seg-Glasser_den-91k_stat-mean_timeseries.ptseries.nii'
    try:
        img = nb.load(im_file)
        img_y = img.get_fdata()
        print(img_y.shape)
        subjects.append(sub)
    except:
        print(f'missing {sub}')
print(f'loaded {len(subjects)} subjects')
#return(subjects)

(750, 360)
(750, 360)
(750, 360)
(700, 360)
(702, 360)
(750, 360)
(750, 360)
(750, 360)
(737, 360)
(750, 360)
(744, 360)
(750, 360)
(728, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
(750, 360)
loaded 28 subjects


In [28]:
# pilots.to_csv('../data/pilots_ru_dm.csv', index=False)

In [33]:
# pilots['Identifiers_y'].to_csv('pilots_ru_dm_list.txt', header=False, index=False)

### make a new pilot selection; remove the current pilots from df

In [20]:
# Load the data into a pandas dataframe
df = dx_final_ru[['Identifiers', 'Age', 'Sex', 'class','imaging_site']]



with open('pilots_ru_dm_list.txt', 'r') as file:
    file_list = [line.strip() for line in file]


df = dx_final_ru[~dx_final_ru['Identifiers'].isin(file_list)]




#df = df.loc[(df['imaging_site'] == 'RU') | (df['imaging_site'] == 'CBIC')]


# Separate cases and controls based on the `Group` variable
cases = df[df['class'] == 0]
controls = df[df['class'] == 1]

# Rename the variables for cases and controls to distinguish them
cases = cases.rename(columns={'Age': 'Age_case'})
controls = controls.rename(columns={'Age': 'Age_control'})

# Join the cases and controls based on age and sex
matched = pd.merge(cases, controls, on=['Sex'])
matched = matched[(matched['Age_control'] - 1 <= matched['Age_case']) & (matched['Age_case'] <= matched['Age_control'] + 1)]

# Randomly select one match if there are more
matched = matched.sample(frac=1).groupby('Identifiers_x').head(1)
matched = matched.sample(frac=1).groupby('Identifiers_y').head(1)

###
#Now using these matched asd and controls, make a matched pilot subset from the non-control non-asd population
###


# Separate cases and controls based on the `Group` variable
controls = df[df['class'] == -1]

# Rename the variables for cases and controls to distinguish them
#cases = cases.rename(columns={'Age': 'Age_case'})
cases= matched[['Identifiers_x', 'Age_case', 'Sex', 'class_x','imaging_site_x']]
cases.rename(columns={'Identifiers_x': 'Identifiers'}, inplace=True)
cases.rename(columns={'class_x': 'class'}, inplace=True)
cases.rename(columns={'imaging_site_x': 'imaging_site'}, inplace=True)


controls = controls.rename(columns={'Age': 'Age_control'})

# Join the cases and controls based on age and sex
matched2 = pd.merge(cases, controls, on=['Sex'])
matched2 = matched2[(matched2['Age_control'] - 1 <= matched2['Age_case']) & (matched2['Age_case'] <= matched2['Age_control'] + 1)]

# Randomly select one match if there are more
matched2 = matched2.sample(frac=1).groupby('Identifiers_x').head(1)
matched2 = matched2.sample(frac=1).groupby('Identifiers_y').head(1)

pilots = matched2[['Identifiers_y', 'Age_control', 'Sex', 'class_y','imaging_site_y']]

/tmp/ipykernel_837465/2209591310.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases.rename(columns={'Identifiers_x': 'Identifiers'}, inplace=True)
/tmp/ipykernel_837465/2209591310.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases.rename(columns={'class_x': 'class'}, inplace=True)
/tmp/ipykernel_837465/2209591310.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases.rename(columns={'imaging_site_x': 'imaging_site'}

In [22]:
pilots.to_csv('../data/pilots2_ru_dm.csv', index=False)
pilots['Identifiers_y'].to_csv('pilots2_ru_dm_list.txt', header=False, index=False)